In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


# 1. Data Pre-Processing

In [14]:
shortlist = list(pd.read_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/shortlist_for_ML_Opti_0d1.csv")['features'])

In [15]:
df_ROI_for_ML_Opti_train = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_cbMSI_0d1/df0d1_ROI4_for_ML_Opti_train.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_ext = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_cbMSI_0d1/df0d1_ROI4_for_ML_Opti_ext.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_ingested = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_cbMSI_0d1/df0d1_ROI4_for_ML_Opti_ingested.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_FNA = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_cbMSI_0d1/df0d1_ROI4_for_ML_Opti_FNA.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_DirectIn = pd.read_csv(r"F:/3_output_raMSIn/3_3_Output_raMSI_HKU_DirectIn/0_cbMSI_0d1/df0d1_ROI4_for_ML_Opti_DI.csv").set_index('pixel_id')[shortlist]

In [16]:
df_pixels_train = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_raMSIn4nonInDI.csv")
pixels_train = list(df_pixels_train["pixel_id"])
df_pixels_ext = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ext_raMSIn4nonInDI.csv")
pixels_ext = list(df_pixels_ext["pixel_id"])
df_pixels_ingested = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ingested_raMSIn4nonInDI.csv")
pixels_ingested = list(df_pixels_ingested["pixel_id"])
df_pixels_FNA = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_FNA_raMSIn4nonInDI.csv")
pixels_FNA = list(df_pixels_FNA["pixel_id"])

In [17]:
df_ROI_for_ML_Opti_train=df_ROI_for_ML_Opti_train.reset_index()
df_ROI_for_ML_Opti_ext=df_ROI_for_ML_Opti_ext.reset_index()
df_ROI_for_ML_Opti_ingested=df_ROI_for_ML_Opti_ingested.reset_index()
df_ROI_for_ML_Opti_FNA=df_ROI_for_ML_Opti_FNA.reset_index()
df_ROI_for_ML_Opti_DirectIn=df_ROI_for_ML_Opti_DirectIn.reset_index()

In [18]:
df_ROI_for_ML_Opti_train = df_ROI_for_ML_Opti_train.loc[df_ROI_for_ML_Opti_train["pixel_id"].isin(pixels_train)].set_index("pixel_id")
df_ROI_for_ML_Opti_ext = df_ROI_for_ML_Opti_ext.loc[df_ROI_for_ML_Opti_ext["pixel_id"].isin(pixels_ext)].set_index("pixel_id")
df_ROI_for_ML_Opti_ingested = df_ROI_for_ML_Opti_ingested.loc[df_ROI_for_ML_Opti_ingested["pixel_id"].isin(pixels_ingested)].set_index("pixel_id")
df_ROI_for_ML_Opti_FNA = df_ROI_for_ML_Opti_FNA.loc[df_ROI_for_ML_Opti_FNA["pixel_id"].isin(pixels_FNA)].set_index("pixel_id")
df_ROI_for_ML_Opti_DirectIn = df_ROI_for_ML_Opti_DirectIn.set_index("pixel_id")

In [19]:
df_ROI_for_ML_Opti_train

,124.0,179.0,195.05,215.0,227.05,233.0,241.1,250.1,253.15,255.1,...,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,2239.475098,5815.817627,1980.816223,4373.655518,1203.201172,3446.257080,1359.884644,10417.782227,9401.526093,25930.855469,...,0.000000,4257.063965,0.0,1094.697510,0.0,2001.672241,31168.623047,15905.403320,9028.520508,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,4155.390625,6869.724792,1921.501221,6860.403076,1378.336182,5497.271484,1761.496948,19751.246094,10752.050781,27755.326172,...,0.000000,4487.459473,0.0,0.000000,0.0,2424.003906,30545.011719,14919.373047,9713.387695,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,1292.693970,4380.000854,590.492004,5703.185852,1561.307861,6164.573242,1269.849487,14473.411133,15996.729156,34604.625000,...,0.000000,3653.516846,0.0,524.351501,0.0,2032.922119,33217.960938,15143.654297,9782.683594,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,1242.931152,4104.772125,1145.876190,4605.494568,1062.043701,5087.580078,1637.160278,14019.504456,12713.678345,31051.693359,...,584.368896,2427.404541,0.0,917.803894,0.0,2229.916992,35512.621094,16859.195312,9062.646484,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,2048.168701,6460.998627,809.635071,3732.185608,1233.044678,2927.715088,1039.213745,9052.196289,10255.392944,26306.203857,...,0.000000,3240.175293,0.0,1397.553223,0.0,2252.401855,33546.597656,17407.480469,8189.253418,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW_106_77,3674.323730,16039.344727,2610.340332,3595.426758,959.914185,3322.557617,0.000000,6830.675293,7581.434570,25921.128906,...,0.000000,1591.155884,0.0,0.000000,0.0,2635.071167,14541.902344,7881.114258,4629.981445,1
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW_40_78,2028.606812,8517.287109,0.000000,2122.539551,964.505127,2835.578125,970.017761,14570.761719,9153.279297,24488.681641,...,0.000000,1300.384399,0.0,1186.961548,0.0,2125.813721,18124.049316,10088.454102,5096.657715,1
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW_41_78,2581.488281,11106.776428,0.000000,2978.173157,1103.738281,4479.292969,0.000000,11204.238281,8237.568604,32790.048584,...,0.000000,2044.695801,0.0,1405.690674,0.0,1770.959473,23411.234375,12572.214844,7795.006836,1


In [20]:
df_ROI_for_ML_Opti_DirectIn

,124.0,179.0,195.05,215.0,227.05,233.0,241.1,250.1,253.15,255.1,...,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_1_1,5.547676e+06,263134.21880,8.075098e+05,393671.69730,7.789545e+05,7.502058e+05,9.057707e+05,1.377530e+06,2.545196e+06,2.419289e+07,...,310379.7813,224169.04690,270663.3750,130111.6641,0.00000,150088.20310,7.726446e+05,367324.1563,211670.87110,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_2_1,8.226763e+06,427196.15630,9.094954e+05,597479.68750,7.373904e+05,9.610786e+05,7.246344e+05,1.996564e+06,2.573555e+06,3.015578e+07,...,470031.5625,352169.21880,665543.9375,357730.1563,167263.57810,235876.03130,1.331586e+06,637292.5625,216978.07810,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_3_1,8.064516e+06,477032.42770,1.055175e+06,361608.53320,7.455225e+05,7.520695e+05,8.470255e+05,1.359076e+06,2.311599e+06,2.450791e+07,...,235263.3047,181263.60940,330792.5000,258239.5156,87975.25781,225600.55080,7.520046e+05,376333.9375,181019.17190,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_4_1,1.021061e+07,555012.00000,1.277905e+06,462620.80470,1.128689e+06,1.006514e+06,1.000048e+06,1.806219e+06,3.198953e+06,3.608494e+07,...,389429.3125,396866.03130,459988.9063,319430.4375,78579.17969,196964.87500,1.451490e+06,592289.4375,350863.87500,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_5_1,1.174849e+07,700037.41800,1.298470e+06,589765.53520,1.345633e+06,1.360949e+06,1.658617e+06,2.311059e+06,3.795908e+06,4.368778e+07,...,630830.5469,356264.90630,874194.7500,464208.0820,257116.73440,319688.31250,1.853836e+06,940595.8125,273320.56250,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_602_1,1.303348e+07,462300.19140,2.549180e+06,611754.18360,4.570672e+05,1.373586e+05,5.074289e+05,3.611135e+04,1.902418e+06,1.771757e+07,...,389582.1094,183160.45310,462036.6250,178496.6914,140198.09380,126309.61330,8.224419e+05,351214.2188,136051.06250,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_603_1,1.040605e+06,283959.47270,5.661782e+05,48820.80664,2.549064e+05,1.385193e+05,2.033246e+05,3.764250e+04,3.294037e+05,2.878697e+06,...,0.0000,0.00000,0.0000,0.0000,0.00000,0.00000,0.000000e+00,0.0000,0.00000,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_604_1,9.098884e+06,613297.69140,2.427616e+06,422294.43750,7.042373e+05,9.743243e+04,4.723518e+05,0.000000e+00,1.476991e+06,1.397649e+07,...,273311.6250,55146.58594,220301.8281,181214.5469,0.00000,63270.16406,5.020935e+05,265960.1875,116673.78130,1


In [21]:
df_ROI_train = df_ROI_for_ML_Opti_train.reset_index()
df_ROI_ext = df_ROI_for_ML_Opti_ext.reset_index()
df_ROI_ingested = df_ROI_for_ML_Opti_ingested.reset_index()
df_ROI_FNA = df_ROI_for_ML_Opti_FNA.reset_index()
df_ROI_DirectIn = df_ROI_for_ML_Opti_DirectIn.reset_index()

In [22]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_ingested["Sample"] = 0
df_ROI_ingested["Row"] = 0
df_ROI_ingested["Scan"] = 0
df_ROI_ingested_Sample = []
df_ROI_ingested_Row = []
df_ROI_ingested_Scan = []
for i in range(len(df_ROI_ingested["pixel_id"])):
    df_ROI_ingested_Sample.append("_".join(str(df_ROI_ingested["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ingested_Row.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-1]))
    df_ROI_ingested_Scan.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-2]))
df_ROI_ingested["Sample"] = df_ROI_ingested_Sample
df_ROI_ingested["Row"] = df_ROI_ingested_Row
df_ROI_ingested["Scan"] = df_ROI_ingested_Scan
pop_column = df_ROI_ingested.pop('Scan')
df_ROI_ingested.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ingested.pop('Row')
df_ROI_ingested.insert(1, 'Row', pop_column)
pop_column = df_ROI_ingested.pop('Sample')
df_ROI_ingested.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_DirectIn["Sample"] = 0
df_ROI_DirectIn["Row"] = 0
df_ROI_DirectIn["Scan"] = 0
df_ROI_DirectIn_Sample = []
df_ROI_DirectIn_Row = []
df_ROI_DirectIn_Scan = []
for i in range(len(df_ROI_DirectIn["pixel_id"])):
    df_ROI_DirectIn_Sample.append("_".join(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[:-2]))
    df_ROI_DirectIn_Row.append(int(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[-1]))
    df_ROI_DirectIn_Scan.append(int(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[-2]))
df_ROI_DirectIn["Sample"] = df_ROI_DirectIn_Sample
df_ROI_DirectIn["Row"] = df_ROI_DirectIn_Row
df_ROI_DirectIn["Scan"] = df_ROI_DirectIn_Scan
pop_column = df_ROI_DirectIn.pop('Scan')
df_ROI_DirectIn.insert(1, 'Scan', pop_column)
pop_column = df_ROI_DirectIn.pop('Row')
df_ROI_DirectIn.insert(1, 'Row', pop_column)
pop_column = df_ROI_DirectIn.pop('Sample')
df_ROI_DirectIn.insert(1, 'Sample', pop_column)

In [23]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ingested = df_ROI_ingested.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_DirectIn = df_ROI_DirectIn.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [24]:
df_ROI_DirectIn

,pixel_id,Sample,Row,Scan,124.0,179.0,195.05,215.0,227.05,233.0,...,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,1,5.547676e+06,2.631342e+05,8.075098e+05,393671.6973,7.789545e+05,750205.83010,...,310379.78130,224169.0469,270663.3750,130111.6641,0.00000,150088.2031,7.726446e+05,367324.1563,211670.87110,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,8.388430e+06,5.111731e+05,9.598826e+05,541398.2891,1.383415e+06,804839.81250,...,241234.48440,238254.5625,564040.0625,301486.5625,188116.31250,184324.8125,1.380952e+06,592850.5625,223408.31250,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,401,7.615059e+06,4.853768e+05,9.363262e+05,597559.6172,9.428713e+05,952275.11330,...,252651.81250,161097.4844,278442.3125,123080.6797,158555.98440,299925.0313,1.135078e+06,423860.4375,197581.54690,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,402,8.341370e+06,5.522360e+05,1.066840e+06,438218.2617,1.052338e+06,772144.18750,...,383306.92190,248052.3438,563859.2500,262979.0859,192461.10550,227635.4063,9.275429e+05,549035.3008,213537.39060,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,403,8.079836e+06,5.573113e+05,1.003882e+06,317669.7188,1.251673e+06,972056.10550,...,404572.92970,335359.9063,614402.0000,314712.2500,72634.45313,303954.9844,1.054263e+06,542855.9883,211347.70310,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,203,5.033229e+05,1.964028e+05,2.703565e+05,0.0000,1.387235e+05,133731.40630,...,0.00000,0.0000,0.0000,0.0000,0.00000,0.0000,0.000000e+00,0.0000,0.00000,1
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,204,6.557098e+06,3.974220e+05,1.354895e+06,145100.5146,4.406531e+05,163806.46190,...,14078.69727,0.0000,0.0000,0.0000,0.00000,0.0000,1.634594e+04,0.0000,0.00000,1
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,205,1.975016e+07,1.120069e+06,3.534104e+06,796009.3181,8.534119e+05,231828.87890,...,150236.95510,115989.1719,259049.7813,171785.4453,66982.69531,110686.8965,3.841148e+05,163241.2188,88670.65625,1
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,207,8.674445e+05,1.823263e+05,4.274579e+05,0.0000,1.469240e+05,165259.34380,...,0.00000,0.0000,0.0000,0.0000,0.00000,0.0000,0.000000e+00,0.0000,0.00000,1


In [25]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [26]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [27]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI"]

In [28]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [29]:
def ROI_Extract(sample_):
    TOP ="885.55"
    preview = sample_[[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)
    vmin_list = list(preview.describe().iloc[-3])
    vmin_ = vmin_list[0]
    sample_ = sample_.loc[sample_["885.55"] > vmin_].reset_index().drop(columns=["index"])

    return sample_

In [30]:
sample_ROI_list_DirectIn_list = []

for sample in sample_ROI_list_DirectIn:
    
    df_ROI = ROI_Extract(sample_=sample)
    sample_ROI_list_DirectIn_list.append(df_ROI)

C:\Users\user\AppData\Local\Temp\ipykernel_34472\797296091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preview[i] = preview[i].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_34472\797296091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preview[i] = preview[i].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_34472\797296091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [31]:
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = sample_ROI_list_DirectIn_list[0]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = sample_ROI_list_DirectIn_list[1]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = sample_ROI_list_DirectIn_list[2]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = sample_ROI_list_DirectIn_list[3]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = sample_ROI_list_DirectIn_list[4]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = sample_ROI_list_DirectIn_list[5]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = sample_ROI_list_DirectIn_list[6]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = sample_ROI_list_DirectIn_list[7]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = sample_ROI_list_DirectIn_list[8]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = sample_ROI_list_DirectIn_list[9]

In [32]:
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI

,pixel_id,Sample,Row,Scan,124.0,179.0,195.05,215.0,227.05,233.0,...,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,2,8226763.0,427196.1563,9.094954e+05,597479.6875,7.373904e+05,9.610786e+05,...,470031.5625,352169.2188,6.655439e+05,357730.1563,167263.57810,235876.0313,1331585.625,6.372926e+05,216978.0781,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,4,10210607.0,555012.0000,1.277905e+06,462620.8047,1.128689e+06,1.006514e+06,...,389429.3125,396866.0313,4.599889e+05,319430.4375,78579.17969,196964.8750,1451490.000,5.922894e+05,350863.8750,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,5,11748486.8,700037.4180,1.298470e+06,589765.5352,1.345633e+06,1.360949e+06,...,630830.5469,356264.9063,8.741948e+05,464208.0820,257116.73440,319688.3125,1853836.250,9.405958e+05,273320.5625,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,9,11873774.0,524780.1875,1.397608e+06,504963.4805,1.302030e+06,9.522283e+05,...,458401.0313,306777.6875,5.200097e+05,357239.9063,180495.23440,345452.5625,1594346.000,7.149868e+05,310046.5938,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,11,10862974.0,831847.2500,1.455680e+06,430475.9063,1.239308e+06,1.224831e+06,...,416463.6250,254745.5156,5.643072e+05,277870.4375,192787.12500,246124.2188,1256030.875,6.319585e+05,204829.1563,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,595,9092751.0,738350.8125,1.218100e+06,802871.8086,1.867512e+06,8.988187e+05,...,475201.4688,348169.3438,1.377972e+06,686636.3125,344897.65630,225340.1875,1579836.875,1.097872e+06,339542.7500,0
297,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,596,10097616.0,630386.0430,1.002128e+06,393550.9375,1.522870e+06,9.260904e+05,...,467592.3438,349874.2500,6.244413e+05,352029.1875,235178.15630,270488.1250,1550177.125,6.325014e+05,268800.0938,0
298,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,598,10767637.0,656517.6211,1.173724e+06,500021.6563,1.366018e+06,9.850924e+05,...,351638.8438,254313.5625,8.760864e+05,534944.8125,270689.93750,271963.9063,1346329.875,7.646786e+05,237992.1563,0
299,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,600,10176433.0,458297.6250,9.809907e+05,327043.7500,1.137197e+06,8.701148e+05,...,582495.0000,491605.6563,8.865479e+05,471787.3945,186632.43750,273641.5000,1440054.750,9.782394e+05,301364.7188,0


In [33]:
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI

,pixel_id,Sample,Row,Scan,124.0,179.0,195.05,215.0,227.05,233.0,...,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55,type
0,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,1,12176738.00,318820.3398,1815148.445,291765.35940,499426.1758,232164.06640,...,520098.1094,384737.87500,2564622.250,1.638535e+06,735014.5000,258773.14060,1.392550e+06,753209.3750,322619.65630,1
1,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,2,14135779.00,269538.7500,2595538.523,339487.87890,590570.6250,251777.07810,...,718009.4375,205010.51560,1715896.375,9.595734e+05,592634.5000,197198.89060,1.173196e+06,534890.7500,58295.61328,1
2,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,11,11727423.60,290527.4688,2150375.684,473564.82810,568018.0430,193036.29690,...,490062.4219,203869.42190,1845380.375,1.117949e+06,555001.2773,147470.82810,8.755424e+05,527000.5000,136271.57810,1
3,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,14,11356945.19,328588.1953,1654949.670,184270.09380,491268.5742,52518.69141,...,328166.0313,157180.79690,1265864.500,4.550678e+05,351378.7188,99370.13672,7.414910e+05,286182.3438,48463.08984,1
4,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,15,13524384.00,351920.8125,2091648.734,334717.39840,673975.2344,216187.98440,...,397795.3320,53684.16797,715738.500,5.573918e+05,196756.5781,105475.18750,7.550344e+05,405735.1875,118373.39060,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,589,8563978.00,222005.7539,1501251.012,130262.14450,471179.3945,322495.52340,...,501994.6719,61469.36328,1339230.625,7.732741e+05,560152.6250,238148.66410,8.186472e+05,383421.5313,230939.51560,1
299,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,591,11366729.00,177693.4570,1567208.469,99632.72266,378515.6855,176942.71480,...,450851.2344,95151.50781,917663.250,5.388859e+05,289344.2500,42115.34375,7.457924e+05,299341.0000,143624.53130,1
300,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,602,10701871.48,248095.2813,1667680.805,195734.64060,523572.0898,185168.98440,...,579655.9844,134935.50000,1808775.500,8.812539e+05,536243.5000,230953.40630,9.289883e+05,502995.8438,181200.14060,1
301,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,604,12146854.00,191779.1602,1989360.238,218544.22660,504580.9609,224908.42190,...,577979.4219,302721.81250,2496260.750,1.154585e+06,736308.3750,239356.60550,1.370530e+06,650568.5625,293856.18750,1


In [34]:
df_ROI_for_ML_Opti_DirectIn = pd.DataFrame()
for df in sample_ROI_list_DirectIn_list:
    df_ROI_for_ML_Opti_DirectIn = pd.concat([df_ROI_for_ML_Opti_DirectIn, df], join="outer", ignore_index=True)

In [35]:
df_ROI_for_ML_Opti_DirectIn=df_ROI_for_ML_Opti_DirectIn.drop(columns=["Sample", "Row", "Scan"]).set_index("pixel_id")

In [36]:
df_ROI_for_ML_Opti_DirectIn

,124.0,179.0,195.05,215.0,227.05,233.0,241.1,250.1,253.15,255.1,...,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_2_1,8.226763e+06,427196.1563,9.094954e+05,5.974797e+05,7.373904e+05,9.610786e+05,7.246344e+05,1.996564e+06,2573554.504,30155780.19,...,470031.5625,352169.2188,6.655439e+05,357730.1563,167263.57810,235876.0313,1331585.625,6.372926e+05,216978.0781,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_4_1,1.021061e+07,555012.0000,1.277905e+06,4.626208e+05,1.128689e+06,1.006514e+06,1.000048e+06,1.806219e+06,3198952.559,36084938.89,...,389429.3125,396866.0313,4.599889e+05,319430.4375,78579.17969,196964.8750,1451490.000,5.922894e+05,350863.8750,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_5_1,1.174849e+07,700037.4180,1.298470e+06,5.897655e+05,1.345633e+06,1.360949e+06,1.658617e+06,2.311059e+06,3795908.285,43687777.97,...,630830.5469,356264.9063,8.741948e+05,464208.0820,257116.73440,319688.3125,1853836.250,9.405958e+05,273320.5625,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_9_1,1.187377e+07,524780.1875,1.397608e+06,5.049635e+05,1.302030e+06,9.522283e+05,1.113740e+06,1.988560e+06,3537040.367,39066860.93,...,458401.0313,306777.6875,5.200097e+05,357239.9063,180495.23440,345452.5625,1594346.000,7.149868e+05,310046.5938,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_11_1,1.086297e+07,831847.2500,1.455680e+06,4.304759e+05,1.239308e+06,1.224831e+06,1.195734e+06,2.105050e+06,4171967.988,40054083.68,...,416463.6250,254745.5156,5.643072e+05,277870.4375,192787.12500,246124.2188,1256030.875,6.319585e+05,204829.1563,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_594_1,1.424341e+07,557954.5000,3.205326e+06,7.968420e+05,5.504298e+05,2.192356e+05,5.821668e+05,0.000000e+00,2773538.500,34466786.08,...,652214.2813,386555.9688,1.163364e+06,410293.7500,284302.46880,340221.1563,1470005.250,7.135132e+05,518926.6875,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_595_1,7.488518e+06,650687.9844,2.885710e+06,5.123862e+05,7.044434e+05,1.970331e+05,6.064224e+05,7.675149e+04,1156182.730,10321883.44,...,600926.0625,335784.5938,8.114948e+05,606009.3281,353163.93750,254814.9063,2130823.750,1.099195e+06,480074.5352,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_596_1,1.945915e+07,710140.3594,3.285616e+06,7.572119e+05,4.459014e+05,1.700947e+05,5.801571e+05,0.000000e+00,1938685.125,23262511.81,...,358901.5625,541987.1875,5.276537e+05,317100.6016,193192.62500,244214.9688,1115554.000,5.783931e+05,258386.8125,1


In [37]:
df_ROI_for_ML_Opti_train.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_ROI_for_ML_Opti_train_0d1.csv")
df_ROI_for_ML_Opti_ext.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_ROI_for_ML_Opti_ext_0d1.csv")
df_ROI_for_ML_Opti_ingested.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_ROI_for_ML_Opti_ingested_0d1.csv")
df_ROI_for_ML_Opti_FNA.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_ROI_for_ML_Opti_FNA_0d1.csv")
df_ROI_for_ML_Opti_DirectIn.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_ROI_for_ML_Opti_DirectIn_0d1.csv")

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [38]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

## 2.2. PCA Analysis

In [51]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_DirectIn):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_DirectIn.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/PCA_DirectIn_0d1.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [52]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [53]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

In [54]:
X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti_ingested)

In [55]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [56]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [57]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [58]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [59]:
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[df_ROI_for_ML_Opti_DirectIn.columns[:-1]]

In [60]:
y_DirectIn = df_ROI_for_ML_Opti_DirectIn[df_ROI_for_ML_Opti_DirectIn.columns[-1]]

## 3.2 Machine Learning Modeling

In [61]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.2. XBG

In [62]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings
mpl.rcParams['figure.dpi'] = 300

In [63]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal= X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2)
    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set AUROC:",metrics.roc_auc_score(yTrain, X_train_predictions))
    print(classification_report(yTrain,X_train_predictions))
    print(confusion_matrix(yTrain,X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_ROC_train_shisted.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val = model_xgb.predict(XVal)
    print("Validation Set AUROC:",metrics.roc_auc_score(yVal, predictions_val))
    print(classification_report(yVal,predictions_val))
    print(confusion_matrix(yVal,predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_ROC_val_shoisted.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test set
    predictions_test = model_xgb.predict(XTest)
    print("Test Set AUROC:",metrics.roc_auc_score(yTest, predictions_test))
    print(classification_report(yTest,predictions_test))
    print(confusion_matrix(yTest,predictions_test))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_test, tpr_test, _ = metrics.roc_curve(yTest, predictions_test)
    plt.plot(fpr_test,tpr_test)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_ROC_test_shosted.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the important features (top 20 are shown)
    feature_imp = pd.Series(model_xgb.feature_importances_,index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp[:25], y=feature_imp[:25].index)
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_TopFeatures_stlisted.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti,x = feature_imp[:20].index,color= 'type')
    #fig.show()
    
    return x, y

In [64]:
coeff_XGB, TOPfeatures_XGB = XGBoost_func()

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.49475	validation_1-auc:0.50527
[1]	validation_0-auc:0.48118	validation_1-auc:0.49572
Training Set AUROC: 0.9281020664806092
              precision    recall  f1-score   support

           0       0.95      0.91      0.93     50392
           1       0.90      0.95      0.93     46643

    accuracy                           0.93     97035
   macro avg       0.93      0.93      0.93     97035
weighted avg       0.93      0.93      0.93     97035

[[45675  4717]
 [ 2341 44302]]
Validation Set AUROC: 0.4947506156469327
              precision    recall  f1-score   support

           0       0.50      0.97      0.66     44540
           1       0.40      0.02      0.04     44161

    accuracy                           0.50     88701
   macro avg       0.45      0.49      0.35     88701
weighted avg       0.45      0.50      0.35     88701

[[43087  1453]
 [43184   977]]
Test Set AUROC: 0.5052709302655315
              precision    recall  f1-score   support

      

In [65]:
coeff_XGB  # coeff

311.15    0.444492
269.25    0.222633
435.2     0.068112
301.25    0.042740
241.1     0.036986
            ...   
325.2     0.000000
179.0     0.000000
329.25    0.000000
351.05    0.000000
887.55    0.000000
Length: 84, dtype: float32

In [66]:
TOPfeatures_XGB  # features

Index(['311.15', '269.25', '435.2', '301.25', '241.1', '738.55', '857.55',
       '867.55', '299.1', '267.1', '794.55', '337.2', '339.1', '233.0',
       '328.15', '124.0', '279.1', '666.5', '353.2', '773.55', '250.1',
       '215.0', '311.95', '763.6', '480.35', '266.15', '277.05', '293.05',
       '760.6', '303.15', '748.55', '592.9', '509.4', '886.55', '885.55',
       '514.3', '883.55', '865.55', '515.25', '863.05', '563.95', '602.45',
       '747.5', '790.5', '770.55', '665.0', '665.8', '767.55', '766.55',
       '762.55', '717.45', '485.25', '327.2', '438.95', '284.25', '195.05',
       '227.05', '253.15', '255.1', '256.25', '265.05', '269.0', '271.15',
       '280.25', '281.1', '282.05', '282.25', '283.1', '285.25', '436.15',
       '295.1', '296.25', '297.1', '297.95', '298.2', '304.25', '305.15',
       '309.2', '313.15', '325.2', '179.0', '329.25', '351.05', '887.55'],
      dtype='object')

In [67]:
len(TOPfeatures_XGB)

84

In [68]:
TOP_features_XGB  # old features (dont run)

NameError: name 'TOP_features_XGB' is not defined

## 3.3. Sensitivity

In [69]:
def df_maker(features_list, df= df_ROI_for_ML_Opti_ingested):
    df_sensitivity = df.T.T.drop(columns=list(df.columns))
    for fea in features_list:
        df_sensitivity[fea] = df[fea]
    df_sensitivity["type"] = df["type"]
    return df_sensitivity

In [70]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

### 3.3.2. XGB

In [71]:
def XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2
    )
    
    #model_xgb.fit(XTrain, yTrain)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    fit_params = {'eval_set': [[XVal, yVal], [XTest, yTest]]}

    scores_XGB = cross_val_score(model_xgb, XTrain, yTrain, cv=5, 
                                 scoring = 'roc_auc',
                                 fit_params = fit_params)
    
    print(scores_XGB.mean())
    print(scores_XGB.std())
    X_train_predictions = model_xgb.predict(XTrain)
    predictions_val = model_xgb.predict(XVal)
    predictions_test = model_xgb.predict(XTest)
    
    a_f1 = f1_score(yTrain, X_train_predictions, pos_label=1)
    b_f1 = f1_score(yVal, predictions_val, pos_label=1)
    c_f1 = f1_score(yTest, predictions_test, pos_label=1)
    a_auc = metrics.roc_auc_score(yTrain, X_train_predictions)
    b_auc = metrics.roc_auc_score(yVal, predictions_val)
    c_auc = metrics.roc_auc_score(yTest, predictions_test)

    cv_acr_XGB = scores_XGB.mean()
    cv_std_XGB = scores_XGB.std()

    return cv_acr_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc

In [72]:
X_ingested

,124.0,179.0,195.05,215.0,227.05,233.0,241.1,250.1,253.15,255.1,...,790.5,794.55,857.55,863.05,865.55,867.55,883.55,885.55,886.55,887.55
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,2239.475098,5815.817627,1980.816223,4373.655518,1203.201172,3446.257080,1359.884644,10417.782227,9401.526093,25930.855469,...,1250.714233,0.000000,4257.063965,0.0,1094.697510,0.0,2001.672241,31168.623047,15905.403320,9028.520508
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,4155.390625,6869.724792,1921.501221,6860.403076,1378.336182,5497.271484,1761.496948,19751.246094,10752.050781,27755.326172,...,1064.463623,0.000000,4487.459473,0.0,0.000000,0.0,2424.003906,30545.011719,14919.373047,9713.387695
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,1292.693970,4380.000854,590.492004,5703.185852,1561.307861,6164.573242,1269.849487,14473.411133,15996.729156,34604.625000,...,820.255554,0.000000,3653.516846,0.0,524.351501,0.0,2032.922119,33217.960938,15143.654297,9782.683594
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,1242.931152,4104.772125,1145.876190,4605.494568,1062.043701,5087.580078,1637.160278,14019.504456,12713.678345,31051.693359,...,1733.640259,584.368896,2427.404541,0.0,917.803894,0.0,2229.916992,35512.621094,16859.195312,9062.646484
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,2048.168701,6460.998627,809.635071,3732.185608,1233.044678,2927.715088,1039.213745,9052.196289,10255.392944,26306.203857,...,1646.036011,0.000000,3240.175293,0.0,1397.553223,0.0,2252.401855,33546.597656,17407.480469,8189.253418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_37_30,1540.501801,4980.301453,4499.848679,5971.031006,6698.260986,8929.067139,1575.619995,9511.707031,910.233734,19202.873169,...,0.000000,0.000000,2976.753021,0.0,0.000000,0.0,1848.187958,6123.190430,2633.123535,1756.053955
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_38_30,1638.716125,4683.008606,3376.917313,5607.661469,5558.725098,9923.133057,1861.503571,10045.843567,1019.449829,22287.158051,...,430.207520,0.000000,2466.429230,0.0,529.190857,0.0,3300.213013,20750.964844,10420.350586,5912.117676
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_39_30,1440.233551,5365.541992,2829.933990,4617.527237,6580.284180,8872.961670,1618.466858,9213.754883,311.672241,19093.204529,...,739.540833,0.000000,3116.599670,0.0,0.000000,0.0,1651.947235,12533.286133,7951.717987,3264.151123


In [73]:
len(list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns))

84

In [74]:
cv_auc_his_XGB = []
cv_change_XGB = [0]
cv_std_his_XGB = []
a_f1_ = []
b_f1_ = []
c_f1_ = []
a_auc_ = []
b_auc_ = []
c_auc_ = []
f1_change_XGB = [0]
auc_change_XGB = [0]
f_ = [""]

features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)

cv_auc_his_XGB.append(cv_auc_XGB)
cv_std_his_XGB.append(cv_std_XGB)

a_f1_.append(a_f1)
b_f1_.append(b_f1)
c_f1_.append(c_f1)

a_auc_.append(a_auc)
b_auc_.append(b_auc)
c_auc_.append(c_auc)

#for i in range(39):
for i in range(len(TOPfeatures_XGB)):
    print(i)
    features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
    features_list.remove(list(TOPfeatures_XGB)[i])
    f_.append(list(TOPfeatures_XGB)[i])
    X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
    X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
    X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
    cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)
    
    cv_auc_his_XGB.append(cv_auc_XGB)
    cv_std_his_XGB.append(cv_std_XGB)

    a_f1_.append(a_f1)
    b_f1_.append(b_f1)
    c_f1_.append(c_f1)
    
    a_auc_.append(a_auc)
    b_auc_.append(b_auc)
    c_auc_.append(c_auc)
    

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.49475	validation_1-auc:0.50527
[1]	validation_0-auc:0.48118	validation_1-auc:0.49572
[2]	validation_0-auc:0.40903	validation_1-auc:0.41886


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.33614	validation_1-auc:0.48252
[1]	validation_0-auc:0.36023	validation_1-auc:0.48153


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



...

[0]	validation_0-auc:0.50048	validation_1-auc:0.47626
[1]	validation_0-auc:0.48747	validation_1-auc:0.31274
[2]	validation_0-auc:0.51631	validation_1-auc:0.50066
[3]	validation_0-auc:0.51040	validation_1-auc:0.47626
[4]	validation_0-auc:0.50092	validation_1-auc:0.47527


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.41487	validation_1-auc:0.50000
[1]	validation_0-auc:0.42657	validation_1-auc:0.50000
[2]	validation_0-auc:0.50416	validation_1-auc:0.50000


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.49969	validation_1-auc:0.47659
[1]	validation_0-auc:0.47491	validation_1-auc:0.47099
[2]	validation_0-auc:0.50016	validation_1-auc:0.50000
[3]	validation_0-auc:0.50754	validation_1-auc:0.49967
0.9135319789545326
0.07636172654786463


In [75]:
df_xgb_TOPfeatures = pd.DataFrame()
df_xgb_TOPfeatures["cv_auc_his_XGB"] = cv_auc_his_XGB
df_xgb_TOPfeatures["cv_std_his_XGB"] = cv_std_his_XGB
df_xgb_TOPfeatures["train_auc"] = a_auc_
df_xgb_TOPfeatures["val_auc"] = b_auc_
df_xgb_TOPfeatures["test_auc"] = c_auc_
df_xgb_TOPfeatures["train_f1"] = a_f1_
df_xgb_TOPfeatures["val_f1"] = b_f1_
df_xgb_TOPfeatures["test_f1"] = c_f1_
df_xgb_TOPfeatures["features"] = f_
df_xgb_TOPfeatures.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_xgb_TOPfeatures.csv")

In [62]:
len(TOP_features_XGB)

NameError: name 'TOP_features_XGB' is not defined

In [76]:
cv_auc_his_XGB = []
cv_change_XGB = [0]
cv_std_his_XGB = []
a_f1_ = []
b_f1_ = []
c_f1_ = []
a_auc_ = []
b_auc_ = []
c_auc_ = []
f1_change_XGB = [0]
auc_change_XGB = [0]

features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)

cv_auc_his_XGB.append(cv_auc_XGB)
cv_std_his_XGB.append(cv_std_XGB)

a_f1_.append(a_f1)
b_f1_.append(b_f1)
c_f1_.append(c_f1)

a_auc_.append(a_auc)
b_auc_.append(b_auc)
c_auc_.append(c_auc)

for i in range(len(features_list)-1):
    print(i)
    #features_list = list(TOPfeatures_XGB)[0:i+1]
    #features_list = list(TOP_features_XGB)#[0:i+1]
    features_list.remove(list(TOPfeatures_XGB)[-(i+1)])
    
    #df_sensitivity = df_maker(features_list=features_list, df= df_ROI_for_ML_Opti_ingested)
    #X_train, X_test, y_train, y_test = split_dataset(df= df_sensitivity)
    X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
    X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
    X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
    #train_acr, test_acr = XGBoost_sensitivity(XTrain=X_train,XTest=X_test,yTrain=y_train,yTest=y_test)
    cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)
    #acr_his_XGB.append(train_acr)
    cv_auc_his_XGB.append(cv_auc_XGB)
    #acr_his_XGB.append(test_acr)
    cv_std_his_XGB.append(cv_std_XGB)
    #print(acr_his_XGB[-2:])
    a_f1_.append(a_f1)
    b_f1_.append(b_f1)
    c_f1_.append(c_f1)
    a_auc_.append(a_auc)
    b_auc_.append(b_auc)
    c_auc_.append(c_auc)
    try:
        #print(((acr_his_XGB[-2]-acr_his_XGB[-4])/acr_his_XGB[-4])*100)
        cv_change_XGB.append(((cv_auc_his_XGB[-1]-cv_auc_his_XGB[0])/cv_auc_his_XGB[0])*100)
        #print(((acr_his_XGB[-1]-acr_his_XGB[-3])/acr_his_XGB[-3])*100)
        auc_change_XGB.append(((c_auc_[-1]-c_auc_[0])/c_auc_[0])*100)
        f1_change_XGB.append(((c_f1_[-1]-c_f1_[0])/c_f1_[0])*100)
    except:
        pass

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.49475	validation_1-auc:0.50527
[1]	validation_0-auc:0.48118	validation_1-auc:0.49572
[2]	validation_0-auc:0.40903	validation_1-auc:0.41886


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.33614	validation_1-auc:0.48252
[1]	validation_0-auc:0.36023	validation_1-auc:0.48153


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



...

[0]	validation_0-auc:0.71588	validation_1-auc:0.50000
[1]	validation_0-auc:0.43280	validation_1-auc:0.50000
[2]	validation_0-auc:0.49202	validation_1-auc:0.50000
0.796574269294371
0.07669942456307326
82
[0]	validation_0-auc:0.48648	validation_1-auc:0.50000
[1]	validation_0-auc:0.50768	validation_1-auc:0.50000
[0]	validation_0-auc:0.49600	validation_1-auc:0.50000
[1]	validation_0-auc:0.52157	validation_1-auc:0.50000
[0]	validation_0-auc:0.49578	validation_1-auc:0.50000


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[1]	validation_0-auc:0.51435	validation_1-auc:0.50000
[0]	validation_0-auc:0.49226	validation_1-auc:0.50000
[1]	validation_0-auc:0.50356	validation_1-auc:0.50000
[0]	validation_0-auc:0.50013	validation_1-auc:0.50000
[1]	validation_0-auc:0.51256	validation_1-auc:0.50000
[2]	validation_0-auc:0.49138	validation_1-auc:0.50000


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.49315	validation_1-auc:0.50000
[1]	validation_0-auc:0.50685	validation_1-auc:0.50000
[2]	validation_0-auc:0.49315	validation_1-auc:0.50000
0.8184091754860898
0.13170751794806226


In [77]:
df_xgb_TOPfeatures = pd.DataFrame()
df_xgb_TOPfeatures["cv_auc_his_XGB"] = cv_auc_his_XGB
df_xgb_TOPfeatures["cv_std_his_XGB"] = cv_std_his_XGB
df_xgb_TOPfeatures["cv_change_XGB"] = cv_change_XGB
df_xgb_TOPfeatures["train_auc"] = a_auc_
df_xgb_TOPfeatures["val_auc"] = b_auc_
df_xgb_TOPfeatures["test_auc"] = c_auc_
df_xgb_TOPfeatures["train_f1"] = a_f1_
df_xgb_TOPfeatures["val_f1"] = b_f1_
df_xgb_TOPfeatures["test_f1"] = c_f1_
df_xgb_TOPfeatures["c_change_XGB_auc"] = auc_change_XGB
df_xgb_TOPfeatures["c_change_XGB_f1"] = f1_change_XGB
df_xgb_TOPfeatures.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/df_xgb_TOPfeatures_shortlisted.csv")

In [78]:
print(len(cv_auc_his_XGB), len(cv_std_his_XGB), len(cv_change_XGB), len(auc_change_XGB))

84 84 84 84


In [79]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

In [81]:
M_new_vec2 = np.array(cv_auc_his_XGB[::-1])
Sigma_new_vec2 = np.array(cv_std_his_XGB[::-1])

lower_bound2 = M_new_vec2 - Sigma_new_vec2
upper_bound2 = M_new_vec2 + Sigma_new_vec2
x_axis2 = np.arange(84)

plt.plot(x_axis2, M_new_vec2, 'go--', linewidth=0.5, markersize=0.1)
plt.fill_between(x_axis2, lower_bound2, upper_bound2, alpha=.3)
plt.plot(16, cv_auc_his_XGB[::-1][16], marker="o", markersize=5, markeredgecolor="blue", markerfacecolor="red")
plt.text(16, cv_auc_his_XGB[::-1][16], "the 17th feature", fontdict=dict(color="black", alpha=0.8, size=18))
plt.xlabel("Number of Features", size=16)
plt.ylabel("Average AUROC Score", size=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim((0,1))
plt.xlim((1,84))

plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_CV_Sensitivity.tif", bbox_inches = 'tight', dpi = 300)
plt.close()

In [82]:
#beCareful
TOP_features_XGB = list(TOPfeatures_XGB)[:17]

In [83]:
len(TOP_features_XGB)

17

In [84]:
TOP_features_XGB

['311.15',
 '269.25',
 '435.2',
 '301.25',
 '241.1',
 '738.55',
 '857.55',
 '867.55',
 '299.1',
 '267.1',
 '794.55',
 '337.2',
 '339.1',
 '233.0',
 '328.15',
 '124.0',
 '279.1']

In [85]:
features_list_XGB = list(TOP_features_XGB)

In [86]:
df_sensitivity_XGB = df_maker(features_list=features_list_XGB, df= df_ROI_for_ML_Opti_ingested)
X_train, X_test, y_train, y_test = split_dataset(df= df_sensitivity_XGB)

In [87]:
X_ingested = df_ROI_for_ML_Opti_ingested[features_list_XGB]

In [88]:
X_FNA = df_ROI_for_ML_Opti_FNA[features_list_XGB]

In [89]:
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list_XGB]

In [90]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2
    )
    

    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set Accuracy:",metrics.roc_auc_score(yTrain, X_train_predictions))
    print(classification_report(yTrain,X_train_predictions))
    print(confusion_matrix(yTrain,X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_ROC_train2.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val = model_xgb.predict(XVal)
    print("Validation Set Accuracy:",metrics.roc_auc_score(yVal, predictions_val))
    print(classification_report(yVal,predictions_val))
    print(confusion_matrix(yVal,predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_ROC_val2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test set
    predictions_FNA = model_xgb.predict(XTest)
    print("FNA Test Set Accuracy:",metrics.roc_auc_score(yTest, predictions_FNA))
    print(classification_report(yTest,predictions_FNA))
    print(confusion_matrix(yTest,predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_ROC_test2.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the important features (top 20 are shown)
    feature_imp = pd.Series(model_xgb.feature_importances_,index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/XGB_TopFeatures2.tif", bbox_inches = 'tight')
    #plt.show()


In [91]:
XGBoost_func()

[0]	validation_0-auc:0.61629	validation_1-auc:0.54001


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[1]	validation_0-auc:0.32161	validation_1-auc:0.49835
[2]	validation_0-auc:0.37392	validation_1-auc:0.52121
Training Set Accuracy: 0.8799897253956398
              precision    recall  f1-score   support

           0       0.89      0.88      0.88     50392
           1       0.87      0.88      0.88     46643

    accuracy                           0.88     97035
   macro avg       0.88      0.88      0.88     97035
weighted avg       0.88      0.88      0.88     97035

[[44253  6139]
 [ 5513 41130]]
Validation Set Accuracy: 0.6162937426262662
              precision    recall  f1-score   support

           0       0.59      0.77      0.67     44540
           1       0.67      0.46      0.55     44161

    accuracy                           0.62     88701
   macro avg       0.63      0.62      0.61     88701
weighted avg       0.63      0.62      0.61     88701

[[34281 10259]
 [23718 20443]]
FNA Test Set Accuracy: 0.5400125207497617
              precision    recall  f1-score   su

In [92]:
noSTDtop16 = pd.DataFrame()
noSTDtop16["top"] = list(features_list_XGB)
noSTDtop16.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/noSTDtop17.csv")

In [93]:
TOPfeatures_XGB_ = list(pd.read_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/0d1/noSTDtop17.csv")["top"])
TOPfeatures_XGB = []
for ion in TOPfeatures_XGB_:
    TOPfeatures_XGB.append(str(ion))